#### Root-finding algorithms

In [7]:
import math
from sympy import *
import pandas as pd

##### Functions

In [3]:
f = [
  lambda x: math.pow(x,10) - 1, 
  lambda x: math.exp(x) - 3 * x, 
  lambda x: math.pow(x,3) - (9 * x) + 5
  ]

##### Auxiliar Function

In [32]:
def print_results(root, iterations, precision, a, b, func, x):
    print(f'Root: {round(root, 4)}')
    print(f'Iterations: {round(iterations, 4)}')
    print(f'a: {round(a, 4)}')
    print(f'b: {round(b, 4)}')
    print(f'Erro: {round(abs(func(x)) - precision, 4)}')

##### Bissection Method

In [28]:
def bissec(a, b, precision, iterations, f):

  if f(a) * f(b) >= 0:
    print('Invalid interval')
    return None
  
  k = 1
  M = f(a)
  x = (a + b) / 2

  initial_data = {'a': a, 'b': b, 'x': x, 'f(x)': f(x), 'f(a)': M, 'f(a)*f(x)': f(a)*f(x)}
  df = pd.DataFrame(initial_data, index=[1])

  while k < iterations  or abs(f(x)) > precision:
    if M * f(x) > 0:
      a = x
      M = f(a)
    else:
      b = x

    x = (a + b) / 2
    df = df.append({'a': a, 'b': b, 'x': x, 'f(x)': f(x), 'f(a)': f(a), 'f(a)*f(x)': f(a)*f(x)}, ignore_index=True)

    k += 1
  
  print(f'Root: {round(f(x), 4)}, Iterations: {k}')
  return df

 

In [29]:
bissec(0.5, 1, 0.01, 5, f[2])

Root: -0.0099, Iterations: 5


,a,b,x,f(x),f(a),f(a)*f(x)
0,0.5000,1.00000,0.750000,-1.328125,0.625000,-0.830078
1,0.5000,0.75000,0.625000,-0.380859,0.625000,-0.238037
2,0.5000,0.62500,0.562500,0.115479,0.625000,0.072174
3,0.5625,0.62500,0.593750,-0.134430,0.115479,-0.015524
4,0.5625,0.59375,0.578125,-0.009899,0.115479,-0.001143


In [49]:
bissec(0, 1.3, 0.000000001, 5, f)

1 0 1.3 -0.9865 -1.0
2 0.65 1.3 -0.2237 -0.9865
3 0.975 1.3 2.6267 -0.2237
4 0.975 1.1375 0.7285 -0.2237
Root: 1.0156
Iterations: 5
a: 0.975
b: 1.0563
Erro: 0.1677
